In [ ]:
from fortigate_methods import FortigateMethods
import requests
import logging
import urllib3
import sys
import os
sys.path.append(os.path.abspath('../vault'))
from vault_methods import VaultMethods

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

logging.basicConfig(format='%(asctime)s %(message)s',
                    datefmt='%m/%d/%Y %I:%M:%S %p',
                    level=logging.INFO)

if __name__ == '__main__':
    vault_session = VaultMethods()
    vault_session.authenticate()
    firewall_token = vault_session.retrieve_firewall_secret()
    logging.info("Retrieving token")
    if firewall_token:
        logging.info("Retrieved firewall token")
        addr = input("Firewall IP/FQDN-> ")
        port = int(input("Firewall interface port-> "))
        firewall_session = FortigateMethods(addr, port, firewall_token)
        valid_token = firewall_session.check_token_validity()
        if valid_token:
            sdwan_zones = firewall_session.list_sdwan_zones()
            zones = firewall_session.list_zones()
            interfaces = firewall_session.list_interfaces()
            srcintf = ""
            dstinft = ""
            addr = ""
            for zone in range(0,len(zones)):
                logging.info("{0} -> {1}".format(zone, zones[zone]))
            srcintf = int(input("Enter the source zone index-> "))
            srcintf = zones[srcintf]
            logging.info("Source zone: {0}".format(srcintf))
            for zone in range(0,len(sdwan_zones)):
                logging.info("{0} -> {1}".format(zone, sdwan_zones[zone]))
            dstintf = int(input("Enter the destination zone index-> "))
            dstintf = sdwan_zones[dstintf]
            logging.info("Destination zone: {0}".format(dstintf))
            fqdn = str(input("Enter the FQDN to block-> "))
            obj_name = firewall_session.create_fqdn_object(fqdn)
            if obj_name:
                firewall_session.create_policy(srcintf, dstintf, obj_name)
            
